## Import needed modules

In [ ]:
# Standard modules to use and manipulate dataframes
import numpy as np
import pandas as pd
# Graphistry API import # Used to be able to access locations on disk
import graphistry, pathlib
from datetime import datetime
import nest_asyncio

In [ ]:
#General
nest_asyncio.apply()
location = %pwd
csvlocation = location + "\\CSV\\"
#lnx_csv = 
#win_csv = 

In [ ]:
def write_csv_data(df_w, filename):
    df_w.to_csv(csvlocation + filename)
    print(filename," saved!")
    return

In [ ]:
def read_csv_data(filename):
    df_r = pd.read_csv(csvlocation + filename)
    return(df_r)

In [ ]:
def intersect(df):
    #group by anon and if there is only 1 unique table name in the grouping drop it, this would mean the data is only available in that table
    extract_df_clean_filter_dup = df.groupby(by='anon')
    extract_df_clean_filter_dup = extract_df_clean_filter_dup.filter(lambda x: x['Table'].nunique() > 1).reset_index(drop=True)
    groups_anon = extract_df_clean_filter_dup.groupby('anon')['Table.Column'].apply(', '.join).to_dict()
    extract_df_clean_filter_dup['intersect'] = extract_df_clean_filter_dup['anon'].map(groups_anon)
    return(extract_df_clean_filter_dup)

## GRAPH Functions
Colors Osquery to use in graph
- rgb(165, 150, 255); #A596FF
- rgb(0, 18, 95); #00125F maybe a ring omheen?
- #00094c; miss te zwart
- #4a9dff; licht blauw

In [ ]:
def node_decorator(nodes):
    #add color
    #0xFF000000, 0x00FF0000, 0x0000FF00
    nodes['node_color'] = [0xA596FF00 if node_type == 'Table' else 0x00125F00 if node_type == 'Table.Column' else 0x4a9dff00 for node_type in nodes['type']] #0, 4, 5
    #subsitute nodeTitle from Table.Column to Column
    nodes['node_title'] = [n_title.split(".")[1] if n_type == 'Table.Column' else n_title for (n_type,n_title) in zip(nodes['type'],nodes['nodeTitle'])]
    #add fontawesome
    nodes['node_icon'] = ["table" if node_type == 'Table' else "columns" if node_type == 'Table.Column' else "asterisk" for node_type in nodes['type']]

    #change these columns to str otherwise error with api3
    #nodes['nodeTitle'] = nodes['nodeTitle'].astype(str)
    #nodes['node_title'] = nodes['node_title'].astype(str)
    nodes['node_color'] = nodes['node_color'].astype(np.int64) #int)
    return(nodes)

def edge_decorator(edges):
    #print(edges.columns)
    #edges = edges.sort_values(by=['src', 'dst'])
    #edges['hash'] = edges['hash'].astype('str')
    return(edges)

### Get JWT Graphistry Token

In [ ]:
def generate_graphistry_token(api_username,api_pwd): 
    global jwt_token_time
    global jwt_token
    if 'jwt_token_time' in globals(): 
        now_time = datetime.now() 
        diff_time = now_time - jwt_token_time
        if diff_time.seconds >= 3600:
            jwt_token_time = datetime.now() 
            jwt_token = graphistry.ArrowUploader(server_base_path='https://hub.graphistry.com').login(username=api_username, password=api_pwd).token
            #print('New token generated!')
        else:
            pass
            #print('Token still current!')
    else:
        jwt_token_time = datetime.now()
        jwt_token = graphistry.ArrowUploader(server_base_path='https://hub.graphistry.com').login(username=api_username, password=api_pwd).token
        #print('New token generated!')
    return(jwt_token)

In [ ]:
def graphistry_graph(df): 
    api_token = generate_graphistry_token(api_username,api_pwd)
    graphistry.register(api=3, protocol="https", server='hub.graphistry.com', token=api_token, key='')
    g3 = graphistry.hypergraph(df, ['Table', 'Table.Column', 'intersect'], direct=True, drop_edge_attrs=True,
        opts={
            "EDGES": {
                "Table": [ "Table.Column" ],
                "Table.Column": [ "intersect" ]
            }
        })['graph']
    g3 = g3.settings(url_params={'strongGravity': 'true'})

    #add color and other stuff
    g3_1 = g3.nodes(node_decorator(g3._nodes)).edges(edge_decorator(g3._edges))
    #without render
    #workaround to fix 60% error when rendering
    return(g3_1.bind(point_title ='node_title', point_color='node_color', point_label='intersect', source='src2', destination='dst2').edges(g3_1._edges.assign(src2=g3_1._edges['src'], dst2=g3_1._edges['dst'])).plot(render=False))

References:
- https://stackoverflow.com/questions/843277/how-do-i-check-if-a-variable-exists
- https://blog.softhints.com/python-3-subtrack-time/
- https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/